In [12]:
# given a residaul, calculate its AMD in relative units and untis of photons

import numpy as np 
import pandas as pd
import astropy.io.fits as fits
import datetime
import scipy.stats as stats
import os

In [50]:
dt = datetime.datetime.now()
print('Timestamp: ' +  str(dt.strftime("%d") + dt.strftime("%b") + dt.strftime("%Y") + '-' + dt.strftime("%X")))
print()

dir_path = '/mnt_home/azuckerman/BL_APF_DAP/APF_spectra/NDRR_all_apf/'
results_file = pd.read_csv('/mnt_home/azuckerman/BL_APF_DAP/SM_stellar_properties/all_apf/specmatch_results_current.csv')
star_list = os.listdir(dir_path)

df = pd.DataFrame(columns = ['Name','SNR', 'AMD_resid [Relative]', 'AMD_resid [Photons]'])

for star in star_list:  
    try:
        if len(star.split('_')) == 2: name = star.split('_')[0] 
        if len(star.split('_')) == 3: name = star.split('_')[0] + '_' + star.split('_')[1] 

        NDRU_path = dir_path + name + '_NDRR.fits'
        hdu_R = fits.open(NDRU_path)
        wl = hdu_R[2].data
        resid = hdu_R[1].data 

        AMD_resid = stats.median_absolute_deviation(resid)
        SNR = float(results_file[results_file['name'] == name]['SNR'])
        median_order_45 = SNR**2
        AMD_photons = stats.median_absolute_deviation((resid + 1) * median_order_45)

        df = df.append(pd.Series([name, SNR, AMD_resid, AMD_photons], index = df.columns), ignore_index = True)

        #print('Residual AMD in relative units:' + str(AMD_resid))
        #print('Residual AMD in relative units:' + str(AMD_photons))
    except TypeError:
        df = df.append(pd.Series([name, np.nan, np.nan, np.nan], index = df.columns), ignore_index = True)
    


Timestamp: 21Apr2022-11:09:06



In [52]:
df.to_csv('residual_AMDs.csv', index = False)

In [54]:
np.nanmedian(df['AMD_resid [Photons]'])

414.7842679039171